# Quantum-Enhanced Urban Watershed Simulation
## Interactive Notebook

This notebook provides an interactive interface to the quantum watershed simulation system.

**Features:**
- Run simulations with different parameters
- Visualize results in real-time
- Perform custom analyses
- Compare scenarios
- Export results

## Setup and Imports

In [1]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display
import warnings
warnings.filterwarnings('ignore')

# Set matplotlib for inline plotting
%matplotlib inline

# Import simulation components
from quantum_watershed_simulation import (
    QuantumWatershedSimulation,
    SimulationParameters,
    WatershedTopology,
    QuantumInfiltrationSystem,
    WaveEvolution,
    WatershedVisualizer
)

# Analysis tools
from watershed_analysis_examples import (
    flood_risk_analysis,
    analyze_drainage_effectiveness
)

print("✓ All imports successful!")
print(f"Grid size: {SimulationParameters.GRID_SIZE}x{SimulationParameters.GRID_SIZE}")
print(f"Domain: {SimulationParameters.DOMAIN_SIZE}m x {SimulationParameters.DOMAIN_SIZE}m")

✓ All imports successful!
Grid size: 150x150
Domain: 750.0m x 750.0m


## 1. Quick Start: Run a Basic Simulation

In [3]:
# Create simulation with moderate rainfall
sim = QuantumWatershedSimulation(rainfall_type='moderate')

print("Simulation initialized successfully!")
print(f"Rainfall type: moderate ({SimulationParameters.RAINFALL_RATES['moderate']} mm/hour)")
print(f"Total qubits: {sim.params.N_QUBITS_ZONE1 + sim.params.N_QUBITS_ZONE2 + sim.params.N_QUBITS_ZONE3 + sim.params.N_QUBITS_PARK + sim.params.N_QUBITS_DRAINAGE}")


QUANTUM-ENHANCED URBAN WATERSHED SIMULATION

Initializing simulation components...
Generating watershed topology...
  - Generated 19 buildings
  - Generated 5 road segments


AttributeError: 'AerProvider' object has no attribute 'get_simulator'

## 2. Visualize the Watershed Topology

In [ ]:
# Plot elevation map with infrastructure
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Elevation map
im1 = axes[0].imshow(sim.topology.elevation, cmap='terrain', origin='lower')
axes[0].set_title('Elevation Map', fontweight='bold', fontsize=14)
plt.colorbar(im1, ax=axes[0], label='Elevation (m)')

# Surface type map
surface_labels = {0: 'Natural', 1: 'Road', 2: 'Building', 3: 'Sidewalk', 4: 'Park', 5: 'Lakebed'}
im2 = axes[1].imshow(sim.topology.surface_type, cmap='tab10', origin='lower')
axes[1].set_title('Land Use / Surface Type', fontweight='bold', fontsize=14)
plt.colorbar(im2, ax=axes[1], label='Type ID')

# Zone map
zone_labels = {1: 'Hills (Residential)', 2: 'Suburban (Mixed)', 3: 'Urban/Lakebed'}
im3 = axes[2].imshow(sim.topology.zone_mask, cmap='Set3', origin='lower')
axes[2].set_title('Watershed Zones', fontweight='bold', fontsize=14)
plt.colorbar(im3, ax=axes[2], label='Zone ID')

# Add buildings to all plots
for bldg in sim.topology.buildings:
    for ax in axes:
        from matplotlib.patches import Rectangle
        rect = Rectangle((bldg['x'], bldg['y']), bldg['size'], bldg['size'],
                       linewidth=0.5, edgecolor='red', facecolor='none', alpha=0.5)
        ax.add_patch(rect)

plt.tight_layout()
plt.show()

print(f"Total buildings: {len(sim.topology.buildings)}")
print(f"Road segments: {len(sim.topology.roads)}")
print(f"Drainage inlets: {len(sim.quantum_system.drainage_inlets)}")

## 3. Run Short Simulation (First 5 minutes)

In [ ]:
# Run for 300 seconds (5 minutes) - faster for testing
n_steps = int(300 / sim.params.DT)  # 150 steps

print(f"Running {n_steps} timesteps ({300/60:.1f} minutes simulated time)...")

for step in range(n_steps):
    t = step * sim.params.DT
    
    # Add rainfall
    sim.wave_system.add_rainfall('moderate')
    
    # Propagate
    sim.wave_system.propagate_step(sim.params.DT)
    
    # Update quantum state
    sim.quantum_system.update_quantum_state(t)
    
    if step % 20 == 0:
        print(f"Progress: {(step/n_steps)*100:.0f}% (t={t:.1f}s)", end='\r')

print("\nSimulation complete!")

# Show statistics
stats = sim.wave_system.get_statistics()
print("\nCurrent Statistics:")
for key, value in stats.items():
    print(f"  {key}: {value:.6f}")

## 4. Visualize Current State

In [ ]:
# Get water densities
surface_density = sim.wave_system.get_surface_density()
drainage_density = sim.wave_system.get_drainage_density()
total_density = sim.wave_system.get_water_density()

# Create visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Surface water (log scale for visibility)
display_surface = np.log10(surface_density * 1e6 + 1)
im1 = axes[0, 0].imshow(display_surface, cmap='Blues', origin='lower')
axes[0, 0].set_title('Surface Water Distribution', fontweight='bold')
plt.colorbar(im1, ax=axes[0, 0], label='Log(Density)')

# Drainage layer
display_drainage = np.log10(drainage_density * 1e6 + 1)
im2 = axes[0, 1].imshow(display_drainage, cmap='Purples', origin='lower')
axes[0, 1].set_title('Subsurface Drainage Layer', fontweight='bold')
plt.colorbar(im2, ax=axes[0, 1], label='Log(Density)')

# Total water
display_total = np.log10(total_density * 1e6 + 1)
im3 = axes[1, 0].imshow(display_total, cmap='viridis', origin='lower')
axes[1, 0].set_title('Total Water (Surface + Drainage)', fontweight='bold')
plt.colorbar(im3, ax=axes[1, 0], label='Log(Density)')

# Infiltration capacity (quantum-controlled)
infiltration = sim.quantum_system.get_infiltration_map()
im4 = axes[1, 1].imshow(infiltration, cmap='YlOrBr', origin='lower')
axes[1, 1].set_title('Active Infiltration Capacity (Quantum)', fontweight='bold')
plt.colorbar(im4, ax=axes[1, 1], label='Depth (m)')

plt.tight_layout()
plt.show()

## 5. Quantum System State Analysis

In [ ]:
# Analyze quantum register states
print("Quantum Register States:")
print("=" * 60)

registers = [
    ('Zone 1 (Hills)', sim.quantum_system.zone1_state),
    ('Zone 2 (Suburban)', sim.quantum_system.zone2_state),
    ('Zone 3 (Urban)', sim.quantum_system.zone3_state),
    ('Park', sim.quantum_system.park_state),
    ('Drainage System', sim.quantum_system.drainage_state)
]

fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

for idx, (name, state) in enumerate(registers):
    if state is not None:
        active = np.sum(state)
        total = len(state)
        percentage = (active / total) * 100
        
        print(f"\n{name}:")
        print(f"  Total qubits: {total}")
        print(f"  Active (|1⟩): {active} ({percentage:.1f}%)")
        print(f"  Inactive (|0⟩): {total - active} ({100-percentage:.1f}%)")
        print(f"  Bitstring: {''.join(map(str, state))}")
        
        # Visualize
        ax = axes[idx]
        colors = ['lightcoral' if bit == 0 else 'lightgreen' for bit in state]
        ax.bar(range(len(state)), state, color=colors, edgecolor='black')
        ax.set_title(f"{name}\n{active}/{total} active", fontweight='bold')
        ax.set_ylabel('State')
        ax.set_xlabel('Qubit Index')
        ax.set_ylim([-0.1, 1.1])
        ax.set_yticks([0, 1])
        ax.grid(True, alpha=0.3, axis='y')

# Hide unused subplot
axes[5].axis('off')

plt.tight_layout()
plt.show()

## 6. Drainage System Status

In [ ]:
# Analyze drainage effectiveness
analyze_drainage_effectiveness(sim)

## 7. Flood Risk Assessment

In [ ]:
# Generate flood risk map
flood_map = flood_risk_analysis(sim, threshold=0.003)

## 8. Custom Analysis: Water Flow Velocity

In [ ]:
# Compute flow direction based on gradient
grad_x = sim.wave_system.grad_V_x
grad_y = sim.wave_system.grad_V_y
grad_magnitude = np.sqrt(grad_x**2 + grad_y**2)

# Visualize flow patterns
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Gradient magnitude (slope)
im1 = axes[0].imshow(grad_magnitude, cmap='RdYlGn_r', origin='lower')
axes[0].set_title('Terrain Gradient (Slope)', fontweight='bold')
plt.colorbar(im1, ax=axes[0], label='Gradient Magnitude')

# Flow vectors with water density overlay
skip = 8
X, Y = np.meshgrid(np.arange(0, sim.params.GRID_SIZE, skip),
                   np.arange(0, sim.params.GRID_SIZE, skip))
U = -grad_x[::skip, ::skip]
V = -grad_y[::skip, ::skip]

axes[1].imshow(surface_density, cmap='Blues', origin='lower', alpha=0.6)
axes[1].quiver(X, Y, U, V, color='red', alpha=0.7, scale=30, width=0.004)
axes[1].set_title('Water Distribution + Flow Direction', fontweight='bold')

plt.tight_layout()
plt.show()

print(f"Maximum slope: {np.max(grad_magnitude):.2f}")
print(f"Average slope: {np.mean(grad_magnitude):.2f}")

## 9. Zone-by-Zone Water Balance

In [ ]:
# Calculate water accumulation by zone
zone_mask = sim.topology.zone_mask
total_density = sim.wave_system.get_water_density()

zone_water = {}
zone_names = {1: 'Hills (Residential)', 2: 'Suburban (Mixed)', 3: 'Urban/Lakebed'}

for zone_id in [1, 2, 3]:
    mask = (zone_mask == zone_id)
    water_in_zone = np.sum(total_density[mask])
    zone_water[zone_id] = water_in_zone

# Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart
zones = list(zone_names.keys())
labels = [zone_names[z] for z in zones]
values = [zone_water[z] for z in zones]
colors = ['brown', 'green', 'blue']

bars = ax1.bar(labels, values, color=colors, alpha=0.7, edgecolor='black')
ax1.set_ylabel('Total Water Quantity', fontweight='bold')
ax1.set_title('Water Accumulation by Zone', fontweight='bold')
ax1.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar, val in zip(bars, values):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
           f'{val:.5f}', ha='center', va='bottom', fontsize=10)

# Pie chart
ax2.pie(values, labels=labels, colors=colors, autopct='%1.1f%%',
       startangle=90, explode=[0.05, 0.05, 0.05])
ax2.set_title('Water Distribution Percentage', fontweight='bold')

plt.tight_layout()
plt.show()

print("\nZone Water Balance:")
print("=" * 50)
total_water = sum(values)
for zone_id, name in zone_names.items():
    water = zone_water[zone_id]
    percentage = (water / total_water) * 100 if total_water > 0 else 0
    print(f"{name:25s}: {water:.6f} ({percentage:5.1f}%)")

## 10. Compare Multiple Rainfall Scenarios

In [ ]:
# Run quick comparisons (reduced timesteps for speed)
scenarios = ['light', 'moderate', 'heavy', 'extreme']
results = {}

print("Running comparative scenarios...\n")

# Reduce simulation time for faster comparison
compare_steps = 200  # ~400 seconds simulation

for scenario in scenarios:
    print(f"Running {scenario:10s}...", end=' ')
    
    sim_temp = QuantumWatershedSimulation(rainfall_type=scenario)
    
    for step in range(compare_steps):
        t = step * sim_temp.params.DT
        sim_temp.wave_system.add_rainfall(scenario)
        sim_temp.wave_system.propagate_step(sim_temp.params.DT)
        sim_temp.quantum_system.update_quantum_state(t)
    
    results[scenario] = sim_temp.wave_system.get_statistics()
    print("✓")

# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Rainfall Scenario Comparison', fontsize=16, fontweight='bold')

metrics = ['surface_water', 'drainage_water', 'lakebed_accumulation', 'total_infiltrated']
titles = ['Surface Water', 'Drainage Layer', 'Lakebed Collection', 'Infiltration']
colors_list = ['lightblue', 'blue', 'darkblue', 'navy']

for idx, (metric, title) in enumerate(zip(metrics, titles)):
    ax = axes[idx // 2, idx % 2]
    
    values = [results[scenario][metric] for scenario in scenarios]
    bars = ax.bar(scenarios, values, color=colors_list, alpha=0.7, edgecolor='black')
    ax.set_title(title, fontweight='bold')
    ax.set_ylabel('Normalized Quantity')
    ax.grid(True, alpha=0.3, axis='y')
    
    # Value labels
    for bar, val in zip(bars, values):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
               f'{val:.4f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

print("\nComparison complete!")

## 11. Export Results

In [ ]:
# Export data to files
import json

# Save statistics
stats = sim.wave_system.get_statistics()
with open('simulation_statistics.json', 'w') as f:
    json.dump(stats, f, indent=2)
print("✓ Saved statistics to simulation_statistics.json")

# Save water density arrays
np.save('surface_water_density.npy', sim.wave_system.get_surface_density())
np.save('drainage_water_density.npy', sim.wave_system.get_drainage_density())
np.save('total_water_density.npy', sim.wave_system.get_water_density())
print("✓ Saved density arrays to .npy files")

# Save topology
np.save('watershed_elevation.npy', sim.topology.elevation)
np.save('watershed_surface_type.npy', sim.topology.surface_type)
np.save('watershed_zones.npy', sim.topology.zone_mask)
print("✓ Saved topology to .npy files")

print("\nAll results exported successfully!")

## 12. Advanced: 3D Visualization

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

# Create 3D surface plot
fig = plt.figure(figsize=(14, 10))

# 3D elevation with water overlay
ax = fig.add_subplot(111, projection='3d')

# Downsample for performance
skip = 3
x = np.arange(0, sim.params.GRID_SIZE, skip)
y = np.arange(0, sim.params.GRID_SIZE, skip)
X, Y = np.meshgrid(x, y)
Z = sim.topology.elevation[::skip, ::skip]

# Water density as color
water = sim.wave_system.get_surface_density()[::skip, ::skip]
water_normalized = water / (np.max(water) + 1e-10)

# Plot surface
surf = ax.plot_surface(X, Y, Z, facecolors=plt.cm.Blues(water_normalized),
                      linewidth=0, antialiased=True, alpha=0.8, shade=True)

ax.set_xlabel('X Position', fontweight='bold')
ax.set_ylabel('Y Position', fontweight='bold')
ax.set_zlabel('Elevation (m)', fontweight='bold')
ax.set_title('3D Watershed Topography with Water Distribution', fontweight='bold', fontsize=14)

# Add colorbar
mappable = plt.cm.ScalarMappable(cmap=plt.cm.Blues)
mappable.set_array(water_normalized)
fig.colorbar(mappable, ax=ax, label='Water Density (normalized)', shrink=0.5)

plt.show()

print("3D visualization complete!")

## Summary

This notebook demonstrates:
- Creating and configuring quantum watershed simulations
- Running simulations with different parameters
- Analyzing quantum register states
- Visualizing water flow and accumulation
- Performing flood risk assessment
- Comparing multiple scenarios
- Exporting results for further analysis

For more advanced features, see:
- `quantum_watershed_simulation.py` - Main simulation code
- `watershed_analysis_examples.py` - Additional analysis tools
- `README_WATERSHED.md` - Complete documentation